In [1]:
import os
import sys
import django

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "True"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()

In [2]:
default_config = {
    "semester": 1,
    "local_search": {
        "algorithm": "simulated_annealing",
        "config": {
            "neighborhood": {
                "algorithm": "random_swap",
                "random_swap": {
                    "neighborhood_size": 25
                },
                "random_range_swap": {
                    "neighborhood_size_factor": 0.1,
                    "range_size_factor": 0.1
                },
                "distance_swap": {
                    "distance_percentage": 0.1
                },
                "swap": False
            },
            "simulated_annealing": {
                "initial_temperature": 25,
                "cooling_rate": 0.1,
                "max_iteration": 250,
                "max_time": 30,
                "max_iteration_without_improvement": 25,
            },
            "tabu_search": {
                "tabu_list_size": 50,
                "max_iteration": 1000,
                "max_time": 60,
                "max_iteration_without_improvement": 100,
                "max_time_without_improvement": 5
            }
        }
    },
    "algorithm": {
        "algorithm": "genetic_algorithm",
        "config": {
            "max_iteration": 500,
            "population_size": 100,
            "elitism_size": 2,
            "fitness": {
                "group_assignment_conflict": {
                    "max_threshold": 3,
                    "conflict_penalty": 0.5
                },
                "assistant_distribution": {
                    "max_group_threshold": 25,
                    "max_shift_threshold": 6,
                    "group_penalty": 0.25,
                    "shift_penalty": 0.75
                },
                "timeslot_conflict": {
                    "assistant_conflict_penalty": 1,
                    "group_conflict_penalty": 0.5
                },
            },
            "operator": {
                "selection": {
                    "roulette_wheel": True,
                    "tournament": True,
                    "elitism": False,
                    "tournament_size": 10
                },
                "crossover": {
                    "single_point": False,
                    "two_point": False,
                    "uniform": True,
                    "crossover_probability": 0.1,
                    "uniform_probability": 0.5
                },
                "mutation": {
                    "swap": True,
                    "shift": False,
                    "random": True,
                    "mutation_probability": 0.05
                },
                "repair": {
                    "time_slot": True
                }
            }
        }
    }
}




In [3]:
from scheduling_algorithm.data_parser import ModuleData
from scheduling_algorithm.algorithms import (
    GeneticAlgorithm,
    GeneticLocalSearch
)
from scheduling_algorithm.utils.solution_generator import SolutionGenerator
from scheduling_algorithm.factory import WeeklyFactory

In [ ]:
population = WeeklyFactory(4,1).generate_population(10,1)

In [4]:
modules = ModuleData.get_modules_by_semester(1)

In [5]:
# generator = SolutionGenerator.from_data(default_config)
weekly_generator = SolutionGenerator.from_data(default_config)

Loading configuration...
Configuration loaded successfully.
Creating FitnessManager with fitness functions: 
Fitness(name=GroupAssignmentCapacityFitness, max_threshold=3, conflict_penalty=0.5)
Fitness(name=AssistantDistributionFitness, max_group_threshold=25, max_shift_threshold=6, group_penalty=0.25, shift_penalty=0.75)
Fitness(name=TimeslotConflictFitness, assistant_conflict_penalty=1, group_conflict_penalty=0.5)
Configured selection functions:  [Selection(name=RouletteWheelSelection), Selection(name=TournamentSelection)]
Configuring crossover operator:  [Crossover(name=UniformCrossover)]
Configuring mutation operator:  [Mutation(name=SwapMutation), Mutation(name=RandomMutation)]
Configuring repair operator:  [Repair(name=RepairTimeSlot)]
Creating Genetic Algorithm Object from Configuration File
Population Size:  100
Max Iteration:  500


In [6]:
import cProfile as profile
import pstats
from pstats import SortKey

profiler = profile.Profile()
profiler.enable()
solution = weekly_generator.generate_solution_weekly_test()
profiler.disable()

Generating population for module 1 week 1
Iteration: 0, Fittest Chromosome: 20.5
Iteration: 1, Fittest Chromosome: 20.0
Iteration: 2, Fittest Chromosome: 20.0
Iteration: 3, Fittest Chromosome: 20.0
Iteration: 4, Fittest Chromosome: 20.0
Iteration: 5, Fittest Chromosome: 19.5
Iteration: 6, Fittest Chromosome: 18.25
Iteration: 7, Fittest Chromosome: 18.25
Iteration: 8, Fittest Chromosome: 18.25
Iteration: 9, Fittest Chromosome: 17.25
Iteration: 10, Fittest Chromosome: 17.25
Iteration: 11, Fittest Chromosome: 17.25
Iteration: 12, Fittest Chromosome: 17.25
Iteration: 13, Fittest Chromosome: 17.25
Iteration: 14, Fittest Chromosome: 17.25
Iteration: 15, Fittest Chromosome: 17.25
Iteration: 16, Fittest Chromosome: 16.5
Iteration: 17, Fittest Chromosome: 16.25
Iteration: 18, Fittest Chromosome: 16.25
Iteration: 19, Fittest Chromosome: 15.5
Iteration: 20, Fittest Chromosome: 15.5
Iteration: 21, Fittest Chromosome: 15.5
Iteration: 22, Fittest Chromosome: 14.75
Iteration: 23, Fittest Chromosome: 

In [7]:
solution

Chromosome(length=288, fitness=0.0)

In [7]:
weekly_generator.algorithm.log

{'iteration_fitness': [(0, 33.0),
  (1, 32.75),
  (2, 32.5),
  (3, 31.75),
  (4, 31.25),
  (5, 31.25),
  (6, 29.75),
  (7, 29.25),
  (8, 29.0),
  (9, 28.0),
  (10, 28.0),
  (11, 26.5),
  (12, 26.5),
  (13, 26.5),
  (14, 26.5),
  (15, 26.25),
  (16, 25.5),
  (17, 25.25),
  (18, 24.25),
  (19, 23.75),
  (20, 23.75),
  (21, 23.75),
  (22, 23.0),
  (23, 22.25),
  (24, 21.5),
  (25, 21.5),
  (26, 20.75),
  (27, 20.75),
  (28, 20.75),
  (29, 20.75),
  (30, 20.75),
  (31, 20.0),
  (32, 20.0),
  (33, 19.75),
  (34, 19.25),
  (35, 19.25),
  (36, 19.25),
  (37, 19.0),
  (38, 18.25),
  (39, 18.25),
  (40, 18.25),
  (41, 18.0),
  (42, 18.0),
  (43, 17.25),
  (44, 17.25),
  (45, 17.25),
  (46, 17.25),
  (47, 17.25),
  (48, 17.0),
  (49, 16.5),
  (50, 15.75),
  (51, 15.75),
  (52, 15.75),
  (53, 15.0),
  (54, 15.0),
  (55, 15.0),
  (56, 15.0),
  (57, 15.0),
  (58, 15.0),
  (59, 15.0),
  (60, 15.0),
  (61, 14.25),
  (62, 14.25),
  (63, 14.25),
  (64, 14.25),
  (65, 14.0),
  (66, 14.0),
  (67, 14.0),


In [9]:
a= solution.copy()
b = solution.copy()

In [12]:
a[0][0] = 2

In [14]:
a = np.array(a)

False

In [13]:
a == b

False

In [8]:
from scheduling_algorithm.structure.chromosome import Chromosome


In [9]:
gene = []
for chromosome in solution[:72]:
    gene.append({"laboratory": chromosome[0], "module": chromosome[1], "chapter": chromosome[2], "group": chromosome[3], "assistant": chromosome[4], "time_slot": (chromosome[5], chromosome[6], chromosome[7])})
gene2 = []
for chromosome in solution[72:]:
    gene2.append({"laboratory": chromosome[0], "module": chromosome[1], "chapter": chromosome[2], "group": chromosome[3], "assistant": chromosome[4], "time_slot": (chromosome[5], chromosome[6], chromosome[7])})

In [10]:
chromosome_copy = Chromosome(gene)
chromosome_copy2 = Chromosome(gene2)

In [39]:
a = [i for i in range(1, 73)]

In [42]:
chromosome_copy.gene_data['assistant'] = [4, 4, 5, 5, 5, 5, 6, 1, 6, 2, 3, 4, 1, 5, 2, 5, 5, 6, 3, 1, 3, 4,
       1, 5, 1, 2, 1, 5, 6, 5, 3, 4, 6, 4, 2, 1, 1, 2, 3, 3, 2, 5, 1, 3,
       6, 2, 3, 5, 4, 4, 2, 1, 1, 6, 6, 2, 6, 2, 5, 1, 3, 3, 4, 3, 3, 6,
       5, 3, 1, 1, 4, 3]

In [43]:
weekly_generator.algorithm.fitness_manager(chromosome_copy)

18.0

In [133]:
chromosome_copy.gene_data[6]['module']

2

In [134]:
chromosome_copy2.gene_data[6]['module']

1

In [132]:
chromosome_copy.gene_data[6]['module'], chromosome_copy2.gene_data[6]['module'] = chromosome_copy2.gene_data[6]['module'], chromosome_copy.gene_data[6]['module']

In [15]:
import numpy as np
def swap_numpy(chromosome1, chromosome2):
    temp = np.copy(chromosome1)
    chromosome1 = np.copy(chromosome2)
    chromosome2 = temp

In [23]:
a = np.array([1, 2, 3, 4, 5, 6])

In [26]:
a[:3]

array([1, 2, 3])

In [125]:
temp = np.copy(chromosome_copy.gene_data[:6]
chromosome_copy.gene_data[:6] = chromosome_copy2.gene_data[:6]
chromosome_copy2.gene_data[:6] = temp

In [126]:
chromosome_copy.gene_data[6]['module']

array([2, 2, 2, 2, 2, 2])

In [127]:
chromosome_copy2.gene_data[6]['module']

array([2, 2, 2, 2, 2, 2])

In [55]:
array1[3], array2[3] = array2[3], array1[3]

In [56]:
array1

array([ 6,  7,  8,  4, 10])

In [57]:
array2

array([1, 2, 3, 9, 5])

In [8]:
stats = pstats.Stats(profiler).sort_stats(SortKey.TIME)
stats.print_stats()

#profiler file
profiler.dump_stats('profiler2.prof')

         104839371 function calls (104005653 primitive calls) in 128.222 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   367030   31.880    0.000   34.197    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\timeslot_conflict.py:32(__call__)
   183515   23.267    0.000   35.459    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\assistant_distribution.py:23(calculate_penalty)
   177674   16.167    0.000   16.167    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\operator\repair\time_slot_repair.py:25(<listcomp>)
   183515   13.372    0.000   14.481    0.000 d:\Projects\Kuliah\skripsi\jte-lab-timetabling\LabTimetablingAPI\scheduling_algorithm\fitness_function\group_assignment_conflict.py:32(calculate_penalty)
 58719720    8.069    0.000    8.069    0.000 {method

In [2]:
from scheduling_algorithm.factory import WeeklyFactory

In [3]:
population = WeeklyFactory(4,1).generate_population(1, 1)

In [11]:
population[0]['time_slot_date'][0]

12.0

In [12]:
a, b, c = (1, 2, 3)

In [14]:
b

2

In [19]:
timeslotconflict(population[0])

5

In [20]:
assistantdist(population[0])

41

In [9]:
groupassign(population[0])

NameError: name 'groupassign' is not defined

In [22]:
from collections import defaultdict, Counter
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

import numpy as np

#Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
class AssistantDistributionFitness(BaseFitness):
    def __init__(self):
        super().__init__("AssistantDistributionFitness")
        self.max_group_threshold = 200
        self.max_shift_threshold = 50
        self.group_penalty = 1
        self.shift_penalty = 1
        
    def __str__(self):
        message = f"Fitness(name={self.name}, max_group_threshold={self.max_group_threshold}, max_shift_threshold={self.max_shift_threshold}, group_penalty={self.group_penalty}, shift_penalty={self.shift_penalty})"
        return message

    def calculate_penalty(self, modules, assistants, groups, timeslots):
        total_penalty = 0
        for assistant in np.unique(assistants):
            assistant_mask = assistants == assistant
            assistant_modules = modules[assistant_mask]
            assistant_groups = groups[assistant_mask]
            assistant_timeslots = timeslots[assistant_mask]
            
            group_counts = Counter(zip(assistant_modules, assistant_groups))
            shift_counts = Counter(zip(assistant_modules, assistant_timeslots))
            
            #Assuming that all the chromosomes have the same module. If we need to generate the schedule for all the module at once, we need to change the way we calculate the penalty
            group_penalty = max(0, (len(group_counts) - self.max_group_threshold) * self.group_penalty)
            shift_penalty = max(0, (len(shift_counts) - self.max_shift_threshold) * self.shift_penalty)
            total_penalty += group_penalty + shift_penalty
        return total_penalty

In [8]:
from collections import defaultdict
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

class TimeslotConflict(BaseFitness):
    def __init__(self):
        """Fitness function to penalize conflicts in timeslot assignment. (e.g. a group or assistant is assigned to the same timeslot more than once)"""
        super().__init__("TimeslotConflictFitness")
        self.assistant_conflict_penalty = None
        self.group_conflict_penalty = None
        
    def __str__(self):
        message = f"Fitness(name={self.name}, assistant_conflict_penalty={self.assistant_conflict_penalty}, group_conflict_penalty={self.group_conflict_penalty})"
        return message

    def __call__(self, timeslot_dates, timeslot_shifts, entity_ids, chapters, penalty, is_assistant=False):
        total_penalty = 0
        seen_combinations = defaultdict(set)
        
        if is_assistant:
            # Check for conflicts where the same assistant is assigned the same timeslot for different chapters
            for i in range(len(timeslot_dates)):
                combination = (timeslot_dates[i], timeslot_shifts[i], entity_ids[i])
                if combination in seen_combinations and chapters[i] not in seen_combinations[combination]:
                    total_penalty += penalty  # Penalize duplicate
                else:
                    seen_combinations[combination].add(chapters[i])
        else:
            # Check for conflicts where the same group is assigned the same timeslot
            for i in range(len(timeslot_dates)):
                combination = (timeslot_dates[i], timeslot_shifts[i], entity_ids[i])
                if combination in seen_combinations:
                    total_penalty += penalty  # Penalize duplicate
                else:
                    seen_combinations[combination].add(chapters[i])
        
        return total_penalty
    
    def calculate_penalty(self, assistants, groups, chapters, timeslot_dates, timeslot_shifts):
        # Check for assistant conflicts
        assistant_penalty = self(timeslot_dates, timeslot_shifts, assistants, chapters, self.assistant_conflict_penalty, is_assistant=True)
        
        # Check for group conflicts
        group_penalty = self(timeslot_dates, timeslot_shifts, groups, chapters, self.group_conflict_penalty, is_assistant=False)
        
        return assistant_penalty + group_penalty
    
    def configure(self, assistant_conflict_penalty, group_conflict_penalty):
        """Configure the fitness function
        Args:
            assistant_conflict_penalty (int): Penalty for each assistant timeslot conflict
            group_conflict_penalty (int): Penalty for each group timeslot conflict"""
        self.assistant_conflict_penalty = assistant_conflict_penalty
        self.group_conflict_penalty = group_conflict_penalty
        return self


In [19]:
# Define a dummy chromosome with potential conflicts
dummy_chromosome = [
    {"assistant": "Assistant1", "group": "Group1", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group2", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group3", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group4", "chapter": "Chapter2", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group5", "chapter": "Chapter2", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant2", "group": "Group1", "chapter": "Chapter1", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant2", "group": "Group1", "chapter": "Chapter2", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},  # Group conflict
    {"assistant": "Assistant3", "group": "Group1", "chapter": "Chapter1", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group2", "chapter": "Chapter1", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group3", "chapter": "Chapter2", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"}  # Assistant conflict
]

# Extract relevant fields for fitness function calculation
assistants = [gene["assistant"] for gene in dummy_chromosome]
groups = [gene["group"] for gene in dummy_chromosome]
chapters = [gene["chapter"] for gene in dummy_chromosome]
timeslot_dates = [gene["time_slot"] for gene in dummy_chromosome]
timeslot_shifts = [gene["laboratory"] for gene in dummy_chromosome]  # Assuming laboratory as shift for this example

# Configure the TimeslotConflict fitness function
timeslot_conflict_fitness = TimeslotConflict().configure(assistant_conflict_penalty=0, group_conflict_penalty=1)

# Calculate the penalty
penalty = timeslot_conflict_fitness.calculate_penalty(assistants, groups, chapters, timeslot_dates, timeslot_shifts)

# Output the penalty
print(f"Total penalty for timeslot conflicts: {penalty}")


Total penalty for timeslot conflicts: 2


In [18]:
from collections import defaultdict
from scheduling_algorithm.structure.chromosome import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

class GroupAssignmentCapacityFitness(BaseFitness):
    def __init__(self):
        """Calculate penalty for exceeding the maximum number of groups that can be assigned to a single time slot in lab.
        (Maksimal jumlah kelompk yang diajar oleh asisten dalam satu waktu)
        """
        super().__init__("GroupAssignmentCapacityFitness")
        self.max_threshold = None
        self.conflict_penalty = None

    def __str__(self):
        return (f"Fitness(name={self.name}, max_threshold={self.max_threshold}, conflict_penalty={self.conflict_penalty})")

    def calculate_penalty(self, labs, modules, assistants, groups, timeslot_dates, timeslot_shifts):
        assistant_timeslot_count = defaultdict(int)
        total_penalty = 0

        for i in range(len(labs)):
            combination = (labs[i], modules[i], assistants[i], timeslot_dates[i], timeslot_shifts[i])
            assistant_timeslot_count[combination] += 1

        for count in assistant_timeslot_count.values():
            if count > self.max_threshold:
                total_penalty += (count - self.max_threshold) * self.conflict_penalty

        return total_penalty

    def configure(self, max_threshold, conflict_penalty):
        """Configure the fitness function with the max threshold and conflict penalty."""
        self.max_threshold = max_threshold
        self.conflict_penalty = conflict_penalty
        return self

In [22]:
dummy_chromosome = [
    {"assistant": "Assistant1", "group": "Group1", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group2", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group3", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant1", "group": "Group4", "chapter": "Chapter1", "time_slot": "Sunday", "laboratory": "Lab1", "module": "Module1"},  # Exceeds max threshold
    {"assistant": "Assistant2", "group": "Group1", "chapter": "Chapter1", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant2", "group": "Group2", "chapter": "Chapter1", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant2", "group": "Group3", "chapter": "Chapter1", "time_slot": "Monday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group1", "chapter": "Chapter1", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group2", "chapter": "Chapter1", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group3", "chapter": "Chapter1", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"},
    {"assistant": "Assistant3", "group": "Group4", "chapter": "Chapter1", "time_slot": "Tuesday", "laboratory": "Lab1", "module": "Module1"}  # Exceeds max threshold
]
# Extract relevant fields for fitness function calculation
labs = [gene["laboratory"] for gene in dummy_chromosome]
modules = [gene["module"] for gene in dummy_chromosome]
assistants = [gene["assistant"] for gene in dummy_chromosome]
groups = [gene["group"] for gene in dummy_chromosome]
timeslot_dates = [gene["time_slot"] for gene in dummy_chromosome]
timeslot_shifts = [gene["laboratory"] for gene in dummy_chromosome]  # Assuming laboratory as shift for this example
# Configure the GroupAssignmentCapacityFitness fitness function
group_assignment_capacity_fitness = GroupAssignmentCapacityFitness().configure(max_threshold=3, conflict_penalty=10)
# Calculate the penalty
penalty = group_assignment_capacity_fitness.calculate_penalty(labs, modules, assistants, groups, timeslot_dates, timeslot_shifts)
# Output the penalty
print(f"Total penalty for group assignment capacity: {penalty}")

Total penalty for group assignment capacity: 20


In [25]:
from typing import List
from scheduling_algorithm.structure import Chromosome
from scheduling_algorithm.fitness_function.base_fitness import BaseFitness

from collections import defaultdict

class FitnessManager:
    def __init__(self, fitness_functions: List[BaseFitness]):
        self.fitness_functions = fitness_functions

    def __call__(self, chromosome: Chromosome) -> int:
        """Calculate the fitness of a chromosome"""
        labs = chromosome["laboratory"]
        modules = chromosome["module"]
        chapters = chromosome["chapter"]
        timeslots = chromosome["time_slot"]
        groups = chromosome["group"]
        assistants = chromosome["assistant"]
            
        # Calculate total fitness
        total_fitness = 0
        for fitness_function in self.fitness_functions:
            if isinstance(fitness_function, GroupAssignmentCapacityFitness):
                total_fitness += fitness_function.calculate_penalty(labs, modules, groups, timeslots)
            elif isinstance(fitness_function, AssistantDistributionFitness):
                total_fitness += fitness_function.calculate_penalty(modules, assistants, groups, timeslots)
            elif isinstance(fitness_function, TimeslotConflict):
                total_fitness += fitness_function.calculate_penalty(assistants, groups, timeslots)

        return total_fitness

In [26]:
timeslotconflict = FitnessManager([TimeslotConflict()])
assistantdist = FitnessManager([AssistantDistributionFitness()])
groupassign = FitnessManager([GroupAssignmentCapacityFitness()])

In [27]:
timeslotconflict(solution)

225.0

In [28]:
assistantdist(solution)

0

In [29]:
groupassign(solution)

9

In [16]:
from scheduling_algorithm.data_parser import CommonData

In [18]:
CommonData.get_schedule(2,2)

{'Friday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': True,
  'Shift5': False,
  'Shift6': False},
 'Monday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Tuesday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Saturday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False},
 'Thursday': {'Shift1': False,
  'Shift2': False,
  'Shift3': False,
  'Shift4': False,
  'Shift5': False,
  'Shift6': True},
 'Wednesday': {'Shift1': False,
  'Shift2': True,
  'Shift3': True,
  'Shift4': False,
  'Shift5': False,
  'Shift6': False}}

In [19]:
CommonData.get_schedule(2,2)['Saturday']['Shift2']

True

In [8]:
import numpy as np
array = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
array2 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [14]:
import timeit as t
t.timeit('np.column_stack((array, array2))', globals=globals())

5.612801700001

In [15]:
t.timeit('np.concatenate((array.reshape(-1, 1), array2.reshape(-1, 1)), axis=1)', globals=globals())

2.5291220000071917